### Retrival Augmentented Generation Day 2

In [ ]:
%pip install langchain_cohere -q
%pip install spacy -q
#ignore error

In [ ]:
# now you need to run this in a terminal window
# python -m spacy download en_core_web_md
# now restart your kernel

Standard imports for the libraires we will be using in this notebook.  Try to keep your imports in the first cell so this can this code can more easliy be converted into a python program later

In [26]:
import boto3
import pandas as pd
import json
import time
import numpy as np
import pyarrow
import traceback
from langchain.embeddings import BedrockEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.chat_models import BedrockChat
from langchain_core.output_parsers import StrOutputParser


# Create the AWS client for the Bedrock runtime with boto3
aws_client = boto3.client(service_name="bedrock-runtime")

#### Lets define functions that will use various embedding models so we can generate vector embeddings
Spacey

In [2]:
def generate_spacy_vector_embedding(text):
    embedder = SpacyEmbeddings(model_name="en_core_web_md")
    query_embedding = embedder.embed_query(text)

    return(np.array(query_embedding))

Huggingface Bert

In [3]:
def generate_huggingface_vector_embedding(text):
    embeddings = HuggingFaceEmbeddings(model_name="bert-base-uncased")
    embedded_texts = embeddings.embed_query(text_data)
    return(embedded_texts)

Cohere

In [4]:
# send in an array size of one and only return the 0th element
def generate_cohere_vector_embedding(text_data):
    input_type = "clustering"
    truncate = "NONE" # optional
    model_id = "cohere.embed-english-v3" # or "cohere.embed-multilingual-v3"
    
    # Create the JSON payload for the request
    json_params = {
            'texts': [text_data],
            'truncate': truncate, 
            "input_type": input_type
        }
    json_body = json.dumps(json_params)
    params = {'body': json_body, 'modelId': model_id,}
    
    # Invoke the model and print the response
    result = aws_client.invoke_model(**params)
    response = json.loads(result['body'].read().decode())
    return(np.array(response['embeddings'][0]))


Amazon Titan

In [5]:
# Let's generate a dense vector using Amazon Titan with LangChain
def generate_titan_vector_embedding(text):
    #create an Amazon Titan Text Embeddings client
    embeddings_client = BedrockEmbeddings(region_name="us-west-2") 

    #Invoke the model
    embedding = embeddings_client.embed_query(text)
    return(np.array(embedding))



This is the mathmatical formula to calcuate cosine similarity between 2 vectors

In [6]:
def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    similarity = dot_product / (norm_vec1 * norm_vec2)
    return similarity



Now let's generate vectors for simple words and compare how we can compute cosine similarity

#### Persistance of embeddings for later retrieval
 In order to do semantic search and retrieve relevant content we need to store that content for later use.  We can store the embedding in several different persistence technologies.  To start simply let's store the data in memory using pandas dataframe.

In [7]:
def clean_value(value):
    value_str = str(value)
    cleaned_value = ''.join(char for char in value_str if char.isalnum() or char.isspace())
    return cleaned_value

In [8]:
def limit_string_size(x, max_chars=2048):
    # Check if the input is a string
    if isinstance(x, str):
        return x[:max_chars]
    else:
        return x

In [10]:
# clean abstract text
#df = pd.read_csv('data/latest_research_articles.csv')
#df['abstract'] = df['abstract'].apply(clean_value)
# for cohere we need to limit the string size to 2048
#df['abstract'] = df['abstract'].apply(limit_string_size)
#df
dft = pd.read_pickle('data/embedded_df.pkl')

### Advanced Retrieval Techniques
#### HyDE
A technique that optimizes semantic matching requires better semantic context.  What if we generated a document from the query hat better match our stored document.

In [53]:
### Retrieval from embedded sources
#Now that we have a dataframe with embedded content of interest, we can use semantic similarity to retrieve the right data to feed to an LLM

# Given the following query let's generate context that more closely matches the embeded data
query = "What is the latest research for broken ribs in children"

In [32]:
# Generate HyDE context

def generate_hyde_response(query_phrase):
    model_id = "anthropic.claude-3-sonnet-20240229-v1:0"

    model_kwargs =  { 
        "max_tokens": 300,
        "temperature": 1,
        "top_k": 250,
        "top_p": 0.9,
        "stop_sequences": ["\n\nHuman"],
    }

    model = BedrockChat(
        client=aws_client,
        model_id=model_id,
        model_kwargs=model_kwargs,
    )

    human_prompt = "Given the following question \n {query} can you please generate a paragraph of text that answers the question. Be sure to use scientific \
                    medical terminology. Please just include the paragraph in your response."
    messages = [
        ("system", "You are a helpful assistant"),
        ("human", human_prompt),
    ]
    try:
        prompt = ChatPromptTemplate.from_messages(messages)

        chain = prompt | model | StrOutputParser()


        # Send the message content to Claude using Bedrock and get the response
        start_time = time.time()  # Start timing
        # Call Bedrock
        response = chain.invoke({"query": query_phrase})
        end_time = time.time()  # End timing
        print("Claude call took :", end_time - start_time)  # Calculate execution time

        return(response)
    except Exception as e:
        exc_type, exc_value, exc_traceback = traceback.sys.exc_info()
        line_number = exc_traceback.tb_lineno
        print(f"Errort: {exc_type}{exc_value}{exc_traceback} on {line_number}")

In [33]:
print(generate_hyde_response(query))

Claude call took : 3.6869916915893555
Recent studies have shed light on the management and treatment of rib fractures in pediatric patients. The current consensus emphasizes a multidisciplinary approach, combining pharmacological interventions for pain management with non-pharmacological strategies such as respiratory therapy and early mobilization. Nonsteroidal anti-inflammatory drugs (NSAIDs) and opioid analgesics are commonly prescribed to alleviate pain and facilitate recovery. However, emerging research highlights the potential benefits of regional analgesia techniques, including intercostal nerve blocks and thoracic epidural analgesia, in mitigating pain and reducing the need for systemic opioids. Additionally, advancements in imaging modalities, such as ultrasound-guided interventions, have improved the accuracy and safety of these procedures in the pediatric population.


#### Titan Embeddings - SAME No HyDE

In [45]:
# Let's search our records for a good semantic search
query_vector = generate_titan_vector_embedding(query)

results = []
# Iterate over each row in the DataFrame
for index, row in dft.iterrows():
    # Extract the value from the specified column
    article_embedding = row['embedded_abstract']
    results.append((index, cosine_similarity(article_embedding, query_vector)))
    #print (index, value)

results.sort(key=lambda x: x[1], reverse=True)
i = 0
# Print the sorted data
print("Here are a few articles that may match your interest:")
for item in results:
    article_title = dft.iloc[item[0]]['title']
    print(f"Abtract: '{article_title}' with a cosine match of: {item[1]}")
    i=i+1
    if i == 5:
        break

Here are a few articles that may match your interest:
Abtract: 'High sensitivity methods for automated rib fracture detection in pediatric radiographs' with a cosine match of: 0.46759500523692665
Abtract: 'Magnetic resonance imaging based finite element modelling of the proximal femur: a short-term in vivo precision study' with a cosine match of: 0.23057253235100217
Abtract: 'On the crashworthiness analysis of bio-inspired DNA tubes' with a cosine match of: 0.21674978237483608
Abtract: 'Reproduction of forearm rotation dynamic using intensity-based biplane 2D–3D registration matching method' with a cosine match of: 0.19907903320363604
Abtract: 'Propagation of extended fractures by local nucleation and rapid transverse expansion of crack-front distortion' with a cosine match of: 0.19056102046718387


In [42]:

# Let's search our records for a good semantic search
query_vector = generate_titan_vector_embedding(generate_hyde_response(query))
# This is a tuple of the article index and the cosine similarity score
results = []
# Iterate over each row in the DataFrame
for index, row in dft.iterrows():
    # Extract the value from the specified column
    article_embedding = row['embedded_abstract']
    results.append((index, cosine_similarity(article_embedding, query_vector)))
    #print (index, value)

results.sort(key=lambda x: x[1], reverse=True)
i = 0
# Print the sorted data
print("Here are a few articles that may match your interest:")
for item in results:
    # Use the index from the Original dataframe to extract values of interest
    article_title = dft.iloc[item[0]]['title']
    print(f"Abtract: '{article_title}' with a cosine match of: {item[1]}")
    i=i+1
    if i == 5:
        break

Claude call took : 6.943073749542236
Here are a few articles that may match your interest:
Abtract: 'High sensitivity methods for automated rib fracture detection in pediatric radiographs' with a cosine match of: 0.5205147419233009
Abtract: 'AI co-pilot bronchoscope robot' with a cosine match of: 0.24361273445341824
Abtract: 'Non-invasive biomarkers for detecting progression toward hypovolemic cardiovascular instability in a lower body negative pressure model' with a cosine match of: 0.2429005756019053
Abtract: 'Reproduction of forearm rotation dynamic using intensity-based biplane 2D–3D registration matching method' with a cosine match of: 0.23163465008048445
Abtract: 'Magnetic resonance imaging based finite element modelling of the proximal femur: a short-term in vivo precision study' with a cosine match of: 0.22144321379030013


#### Differentiating the retrievals and ranking
 How can we evaluate if the cosine similarities are different enough?  Thresholds aren't consistent enough.  What about Z-Score?

In [43]:
def calculate_zscores(cosine_scores):
    zscores = []
    # Calculate the mean of the sample points
    mean = np.mean(cosine_scores)
    # Calculate the standard deviation of the sample points
    std_deviation = np.std(cosine_scores, ddof=1)  # ddof=1 for sample standard deviation
    # Calculate the z-scores for each sample point
    z_scores = [(x - mean) / std_deviation for x in cosine_scores]

    return z_scores

In [50]:
# grab the cosine_scores from the results tuple
cosine_scores = [item[1] for item in results]
z_scores = calculate_zscores(cosine_scores)
#save the articles that were a good match
articles = ""
i=0
first_z_score = z_scores[0]
for item in results:
    # Use the index from the Original dataframe to extract values of interest
    article_title = dft.iloc[item[0]]['title']
    abstract = dft.iloc[item[0]]['abstract']
    # If the highest Z score is 2x this record then likey not as significant
    if(first_z_score/2)<z_scores[i]:
        print(f"Abtract: '{article_title}' with a cosine match of: {item[1]} and Z-score of: {z_scores[i]}")
        articles = articles + " " + abstract
    i=i+1
    

Abtract: 'High sensitivity methods for automated rib fracture detection in pediatric radiographs' with a cosine match of: 0.46759500523692665 and Z-score of: 10.649061266665777


In [51]:
# Now let's take the records that are greater that 1/2 the top Z-score and send those to the LLM for an answer
def best_answer(data, question):
    model_id = "anthropic.claude-3-sonnet-20240229-v1:0"

    model_kwargs =  { 
        "max_tokens": 2048,
        "temperature": 0.0,
        "top_k": 250,
        "top_p": 0.9,
        "stop_sequences": ["\n\nHuman"],
    }

    model = BedrockChat(
        client=bedrock_runtime,
        model_id=model_id,
        model_kwargs=model_kwargs,
    )

    human_prompt = "You are to answer the question using the data in the following information.  Do not make up your answer, only use \
                    supporting data from the article, If you don't have enough data simply respond, I don't have enough information to answer that question. \
                    given the following article data {data} can you please give a concise answer to the following question. {question}"
    messages = [
        ("system", "You are a helpful assistant that can answer quesitons based on news articles you have been given."),
        ("human", human_prompt),
    ]
    try:
        prompt = ChatPromptTemplate.from_messages(messages)

        chain = prompt | model | StrOutputParser()

        # Chain Invoke
        
    
        # Send the message content to Claude using Bedrock and get the response
        start_time = time.time()  # Start timing
        # Call Bedrock
        response = chain.invoke({"data": data,"question": question})
        end_time = time.time()  # End timing
        #print("Claude call took :", end_time - start_time)  # Calculate execution time

        return(response)
    except Exception as e:
        exc_type, exc_value, exc_traceback = traceback.sys.exc_info()
        line_number = exc_traceback.tb_lineno

        return f"ERROR generating good answer: {exc_type}{exc_value}{exc_traceback} on {line_number}"


In [55]:
print("Here is the best answer I could find about ", query)
print(best_answer(articles, query))

Here is the best answer I could find about  What is the latest research for broken ribs in children
Claude call took : 7.580817699432373
Based on the information provided in the article, the latest research focuses on improving the detection and localization of rib fractures in pediatric chest radiographs using computer vision and machine learning techniques. Specifically:

- The research implemented convolutional neural network (CNN) architectures like RetinaNet and YOLOv5, along with an "avalanche decision" scheme to dynamically adjust detection thresholds, to increase the sensitivity (recall) in detecting rib fractures.

- Multiple image preprocessing techniques and model ensembling were used to further enhance detection performance.

- Using a dataset of 1,109 pediatric chest radiographs manually labeled by radiologists, the best performing model achieved an F2 score of 0.725, approaching the expert inter-reader performance of 0.732.

- The goal was to develop methods that can iden